# Python 2

In [1]:
import numpy as np

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

In [2]:
!ls ~/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/*.lhco

/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run_04.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run_05.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run_06.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run_07.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run_08.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_10.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_11.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_12.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_13.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_14.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_15.lhco
/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_

## Backgrounds

### p p > W $\gamma$ > $\nu$ + l + $\gamma$ (NOT IMPLEMENTED)

In [22]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptoWgamma/'
datFolder = '../data/Wgamma/'

Wgamma_cross = 168.12 #[pb]
Wgamma_runs  = 135
Wgamma_NLO   = 1.27 # k-factor [14 TeV]
Wgamma_init_sim_eve = 100000 * Wjets_runs

Wgamma_init_ev = 0
Wgamma_cut_ev  = 0

Wgamma_photon  = []
Wgamma_jet     = []
Wgamma_lepton  = []
Wgamma_MET     = []
Wgamma_hadinfo = []


for ii in range(1, 1 + Wgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        Wgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:

                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                        if inputevents[i]["MET"][0]["PT"] > 100:

                            # how many jets (total) and total hadronic pT
                            for j in range(inputevents[i].number()["jet"]):
                                if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]


                            # photon info:
                            with open(datFolder + 'photon.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))

                            # jet info:
                            with open(datFolder + 'jet.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                            # lepton info
                            if inputevents[i].number()["electron"] > 0:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                            else:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))

                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder + 'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                            # hadronic info
                            with open(datFolder + 'hadinfo.dat', 'a') as f:
                                f.write('%f %d\n' % (HT_had, num_jets))

                            # total events that passed the cuts
                            Wgamma_cut_ev += 1

print('Num initial events: ', Wgamma_init_ev)
print('Total pasaron los cortes: ', Wgamma_cut_ev)

1
Esta run tuvo 96394 eventos en el lhco
2
Esta run tuvo 96333 eventos en el lhco
3
Esta run tuvo 96331 eventos en el lhco
4
Esta run tuvo 96491 eventos en el lhco
5
Esta run tuvo 96378 eventos en el lhco
6
Esta run tuvo 96527 eventos en el lhco
7
Esta run tuvo 96453 eventos en el lhco
8
Esta run tuvo 96428 eventos en el lhco
9
Esta run tuvo 96467 eventos en el lhco
10
Esta run tuvo 96433 eventos en el lhco
11
Esta run tuvo 96472 eventos en el lhco
12
Esta run tuvo 96347 eventos en el lhco
13
Esta run tuvo 96530 eventos en el lhco
14
Esta run tuvo 96510 eventos en el lhco
15
Esta run tuvo 96353 eventos en el lhco
16
Esta run tuvo 96462 eventos en el lhco
17
Esta run tuvo 96376 eventos en el lhco
18
Esta run tuvo 96341 eventos en el lhco
19
Esta run tuvo 96392 eventos en el lhco
20
Esta run tuvo 96418 eventos en el lhco
21
Esta run tuvo 96464 eventos en el lhco
22
Esta run tuvo 96371 eventos en el lhco
23
Esta run tuvo 96395 eventos en el lhco
24
Esta run tuvo 96402 eventos en el lhco
2

In [24]:
Wgamma_cross = 0.94498
Wgamma_NLO = 1.2

In [25]:
cross       = Wgamma_cross * 1000 * Wgamma_NLO
aceptancia  = Wgamma_cut_ev / (1. * Wgamma_init_sim_eve)
luminosidad = 100

Wgamma_fidcross = cross * aceptancia
Wgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wgamma_fidcross)
print('Events expected: ', Wgamma_expected)

('fiducial cross section: ', 519.2364089297561)
('Events expected: ', 51923.64089297561)


### p p > W jets > $\nu$ l jets

In [ ]:
# PAPER MADGRAPH for the NLO corrections
# https://arxiv.org/pdf/1405.0301.pdf

In [19]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/'
datFolder = '../data/Wjets/'

Wjets_cross = 168.12 #[pb]
Wjets_runs  = 82
Wjets_NLO   = 1.27 # k-factor [14 TeV]
Wjets_init_sim_eve = 100000 * Wjets_runs + 2000000

Wjets_init_ev = 0
Wjets_cut_ev  = 0

Wjets_photon  = []
Wjets_jet     = []
Wjets_lepton  = []
Wjets_MET     = []
Wjets_hadinfo = []


for ii in range(1, 1 + Wjets_runs):
    print(ii)
    if ii not in [19,33]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        Wjets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:

                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                        if inputevents[i]["MET"][0]["PT"] > 100:

                            # how many jets (total) and total hadronic pT
                            for j in range(inputevents[i].number()["jet"]):
                                if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]


                            # photon info:
                            with open(datFolder + 'photon.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))

                            # jet info:
                            with open(datFolder + 'jet.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                            # lepton info
                            if inputevents[i].number()["electron"] > 0:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                            else:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))

                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder + 'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                            # hadronic info
                            with open(datFolder + 'hadinfo.dat', 'a') as f:
                                f.write('%f %d\n' % (HT_had, num_jets))

                            # total events that passed the cuts
                            Wjets_cut_ev += 1

print('Num initial events: ', Wjets_init_ev)
print('Total pasaron los cortes: ', Wjets_cut_ev)

1
Esta run tuvo 54919 eventos en el lhco
2
Esta run tuvo 54896 eventos en el lhco
3
Esta run tuvo 54637 eventos en el lhco
4
Esta run tuvo 54863 eventos en el lhco
5
Esta run tuvo 54743 eventos en el lhco
6
Esta run tuvo 54751 eventos en el lhco
7
Esta run tuvo 54698 eventos en el lhco
8
Esta run tuvo 54635 eventos en el lhco
9
Esta run tuvo 54703 eventos en el lhco
10
Esta run tuvo 54469 eventos en el lhco
11
Esta run tuvo 54639 eventos en el lhco
12
Esta run tuvo 54783 eventos en el lhco
13
Esta run tuvo 54687 eventos en el lhco
14
Esta run tuvo 54490 eventos en el lhco
15
Esta run tuvo 54679 eventos en el lhco
16
Esta run tuvo 54319 eventos en el lhco
17
Esta run tuvo 54348 eventos en el lhco
18
Esta run tuvo 54727 eventos en el lhco
19
20
Esta run tuvo 54607 eventos en el lhco
21
Esta run tuvo 54813 eventos en el lhco
22
Esta run tuvo 54729 eventos en el lhco
23
Esta run tuvo 54470 eventos en el lhco
24
Esta run tuvo 54601 eventos en el lhco
25
Esta run tuvo 54588 eventos en el lhc

In [20]:
cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 615.8786110588235)
('Events expected: ', 61587.861105882344)
